# Parte 4

1. Importacion y configuracion de ChromaDB

In [4]:
ruta_dataset = "wiki_movie_plots_deduped.csv"
import chromadb
from chromadb.utils import embedding_functions
import numpy as np
import pandas as pd
from tqdm import tqdm
import textwrap
import shutil
import os
import time
import gc

2. Función para limpiar ChromaDB

In [5]:
def cleanup_chromadb():
    """
    Limpia la instancia anterior de ChromaDB de manera segura
    """
    persist_directory = f"chroma_db_{int(time.time())}"
    return persist_directory

# Celda 3: Configurar ChromaDB
def setup_chromadb():
    """
    Configura una nueva instancia de ChromaDB con una colección para películas
    """
    persist_directory = cleanup_chromadb()
    
    try:
        # Inicializar el cliente de ChromaDB con persistencia
        client = chromadb.PersistentClient(path=persist_directory)
        
        # Usar el embedding function por defecto
        embedding_function = embedding_functions.DefaultEmbeddingFunction()
        
        # Crear una nueva colección
        collection = client.create_collection(
            name="movies_collection",
            embedding_function=embedding_function,
            metadata={"description": "Colección de películas de Wikipedia con sus tramas"}
        )
        print(f"Nueva colección creada en {persist_directory}")
        
        return collection
    
    except Exception as e:
        print(f"Error al configurar ChromaDB: {str(e)}")
        raise

3. Cargar dataset y se procesa

In [6]:
def load_movie_dataset(ruta_dataset, num_samples=None):
    """
    Carga el dataset de películas desde el CSV
    """
    try:
        # Cargar solo las columnas necesarias
        columns = ['Title', 'Plot', 'Release Year', 'Director', 'Genre', 'Origin/Ethnicity']
        df = pd.read_csv(ruta_dataset, usecols=columns)
        print(f"Dataset cargado exitosamente. Dimensiones: {df.shape}")
        
        # Limpiar valores nulos
        df = df.fillna('')
        
        if num_samples:
            df = df.sample(n=min(num_samples, len(df)), random_state=42)
            print(f"Muestra seleccionada: {len(df)} películas")
        
        # Limpiar memoria
        gc.collect()
        
        return df
    except Exception as e:
        print(f"Error al cargar el dataset: {str(e)}")
        raise

# Celda 5: Procesar un lote de películas
def process_batch(batch_df):
    """
    Procesa un lote de películas
    """
    ids = [f"movie_{idx}" for idx in batch_df.index]
    plots = batch_df['Plot'].astype(str).tolist()
    
    metadatas = batch_df.apply(
        lambda row: {
            "title": str(row['Title']),
            "year": str(row['Release Year']),
            "director": str(row['Director']),
            "genre": str(row['Genre']),
            "origin": str(row['Origin/Ethnicity'])
        }, 
        axis=1
    ).tolist()
    
    return ids, plots, metadatas

# Celda 6: Insertar películas en lotes pequeños
def insert_movies_batch(collection, df, batch_size=10):
    """
    Inserta películas en lotes pequeños en la colección de ChromaDB
    """
    total_batches = len(df) // batch_size + (1 if len(df) % batch_size != 0 else 0)
    
    for i in tqdm(range(total_batches), desc="Insertando películas"):
        try:
            # Procesar solo un lote a la vez
            start_idx = i * batch_size
            end_idx = min((i + 1) * batch_size, len(df))
            batch_df = df.iloc[start_idx:end_idx].copy()
            
            # Procesar el lote
            ids, plots, metadatas = process_batch(batch_df)
            
            # Insertar el lote
            collection.add(
                ids=ids,
                documents=plots,
                metadatas=metadatas
            )
            
            # Limpiar memoria después de cada lote
            del batch_df
            gc.collect()
            
            # Pequeña pausa entre lotes
            time.sleep(0.1)
            
        except Exception as e:
            print(f"Error en el lote {i}: {str(e)}")
            continue

4. Consultar peliculas similares

In [7]:
def query_similar_movies(collection, query_text, n_results=5):
    """
    Busca películas similares basadas en una consulta de texto
    """
    try:
        results = collection.query(
            query_texts=[query_text],
            n_results=n_results
        )
        
        similar_movies = []
        for i in range(len(results['documents'][0])):
            metadata = results['metadatas'][0][i]
            plot = textwrap.fill(results['documents'][0][i], width=80)
            
            similar_movies.append({
                'title': metadata['title'],
                'year': metadata['year'],
                'director': metadata['director'],
                'genre': metadata['genre'],
                'origin': metadata['origin'],
                'plot': plot,
                'distance': results['distances'][0][i]
            })
        
        return similar_movies
    except Exception as e:
        print(f"Error en la búsqueda: {str(e)}")
        return []

5. Funcion Principal para generar resultados

In [8]:
def main():
    try:
        # 1. Cargar el dataset
        print("Cargando dataset...")
        df = load_movie_dataset(ruta_dataset, num_samples=50)  # Reducido a 50 para prueba
        
        # 2. Configurar ChromaDB
        print("Configurando ChromaDB...")
        collection = setup_chromadb()
        
        # 3. Insertar películas
        print("Insertando películas en ChromaDB...")
        insert_movies_batch(collection, df, batch_size=10)  # Reducido a 10 por lote
        
        # 4. Ejemplo de consulta
        print("\nRealizando consulta de ejemplo...")
        query = "una película sobre viajes en el tiempo y paradojas"
        similar_movies = query_similar_movies(collection, query)
        
        # 5. Mostrar resultados
        if similar_movies:
            print("\nPelículas similares encontradas:")
            for movie in similar_movies:
                print("\n" + "="*80)
                print(f"Título: {movie['title']} ({movie['year']})")
                print(f"Director: {movie['director']}")
                print(f"Género: {movie['genre']}")
                print(f"Origen: {movie['origin']}")
                print(f"Distancia: {movie['distance']:.4f}")
                print("\nTrama:")
                print(movie['plot'])
        else:
            print("No se encontraron resultados")
            
    except Exception as e:
        print(f"Error durante la ejecución: {str(e)}")
        raise

if __name__ == "__main__":
    main()


Cargando dataset...
Dataset cargado exitosamente. Dimensiones: (34886, 6)
Muestra seleccionada: 50 películas
Configurando ChromaDB...
Nueva colección creada en chroma_db_1736826284
Insertando películas en ChromaDB...


Insertando películas: 100%|██████████| 5/5 [00:01<00:00,  2.59it/s]


Realizando consulta de ejemplo...

Películas similares encontradas:

Título: Opium and the Kung-Fu Master (1984)
Director: Tang Chia
Género: martial arts / action
Origen: Hong Kong
Distancia: 1.6393

Trama:
Master Tie Qiao San (Ti Lung), the leader of the Ten Tigers of Canton, is the
chief coach of China's militia. At that time, the opium ban was in use, but
public sale of opium was widely available. Many bureaucrats were taking opium, a
trend that Tie followed. As Tie takes opium, his physical skills were declining.
Seeing how Tie's skills are declining, opium store owner Rong Feng (Chen Kuan-
tai) challenges Tie to a public duel. Tie struggles to fight Rong and was in
serious danger until his disciple Lu Guasi (Robert Mak) steps in to rescue him
before dying from his injuries. Seeing his disciple killed by Rong, Tie vows to
seek revenge.

Título: The Slipper and the Rose (1976)
Director: Bryan Forbes
Género: musical
Origen: British
Distancia: 1.7277

Trama:
Prince Edward of Euphrani